In [ ]:
# def impact_estimation(entity, test_1, test_2):

#     query = f"""

#         WITH test_data AS (
#         SELECT
#             entity_id,
#             perseus_client_id,
#             CAST(test_id AS INT64) AS test_id,
#             LOWER(test_variant) AS test_variant
#         FROM `fulfillment-dwh-production.cl.dps_test_users`
#         WHERE entity_id = '{entity}'
#         AND test_id IN ({test_1}, {test_2})  -- Specific test IDs
#     ),

#     pivoted_data AS (
#         SELECT
#             entity_id,
#             perseus_client_id,
#             MAX(CASE WHEN test_id = {test_1} AND lower(test_variant) = "control" THEN 1 ELSE 0 END) AS is_control_A,
#             MAX(CASE WHEN test_id = {test_1} AND lower(test_variant) LIKE '%variation%' THEN 1 ELSE 0 END) AS is_variation_A,
#             MAX(CASE WHEN test_id = {test_2} AND lower(test_variant) = "control" THEN 1 ELSE 0 END) AS is_control_B,
#             MAX(CASE WHEN test_id = {test_2} AND lower(test_variant) LIKE '%variation%' THEN 1 ELSE 0 END) AS is_variation_B
#         FROM test_data
#         GROUP BY entity_id, perseus_client_id
#     ),

#     classified_users AS (
#         SELECT
#             entity_id,
#             perseus_client_id,
#             CASE
#                 WHEN is_control_A = 1 AND is_control_B = 1 THEN 'Control'
#                 WHEN is_variation_A = 1 AND is_control_B = 1 THEN 'A Only'
#                 WHEN is_control_A = 1 AND is_variation_B = 1 THEN 'B Only'
#                 WHEN is_variation_A = 1 AND is_variation_B = 1 THEN 'A+B'
#             END AS user_group
#         FROM pivoted_data
#         WHERE NOT (is_control_A = 1 AND is_variation_A = 1)  
#         AND NOT (is_control_B = 1 AND is_variation_B = 1)
#     ),

#     user_metrics AS (
#         SELECT 
#             m.perseus_client_id,
#             m.entity_id,
#             COUNT(DISTINCT m.order_id) AS total_orders,
#             SUM(m.fully_loaded_gross_profit_eur) AS flgp_eur
#         FROM `fulfillment-dwh-production.cl.dps_test_orders` m
#         WHERE entity_id = '{entity}'
#         AND test_id in ({test_1}, {test_2})  -- Orders only during the test period
#         GROUP BY m.perseus_client_id, m.entity_id
#     ),

#     order_metrics AS (
#         SELECT
#             cu.entity_id,
#             cu.user_group,
#             AVG(COALESCE(uo.total_orders, 0)) AS avg_orders_per_user,
#             AVG(COALESCE(uo.flgp_eur, 0)) AS avg_flgp_per_user,
#             SUM(COALESCE(uo.total_orders, 0)) AS total_orders,
#             SUM(COALESCE(uo.flgp_eur, 0)) AS total_flgp,
#             COUNT(DISTINCT cu.perseus_client_id) AS total_users
#         FROM classified_users cu
#         LEFT JOIN user_metrics uo  
#             ON cu.perseus_client_id = uo.perseus_client_id 
#             AND cu.entity_id = uo.entity_id
#         WHERE cu.user_group IS NOT NULL
#         GROUP BY cu.entity_id, cu.user_group
#     ),

#     did_calculations AS (
#         SELECT 
#             entity_id,

#             ROUND(
#                 COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN avg_orders_per_user ELSE NULL END), 0) 
#             - COALESCE(MAX(CASE WHEN user_group = 'A Only' THEN avg_orders_per_user ELSE NULL END), 0)
#             - COALESCE(MAX(CASE WHEN user_group = 'B Only' THEN avg_orders_per_user ELSE NULL END), 0)
#             + COALESCE(MAX(CASE WHEN user_group = 'Control' THEN avg_orders_per_user ELSE NULL END), 0),
#             4) AS did_interaction_effect_orders,

#             ROUND(
#                 COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN avg_flgp_per_user ELSE NULL END), 0) 
#             - COALESCE(MAX(CASE WHEN user_group = 'A Only' THEN avg_flgp_per_user ELSE NULL END), 0)
#             - COALESCE(MAX(CASE WHEN user_group = 'B Only' THEN avg_flgp_per_user ELSE NULL END), 0)
#             + COALESCE(MAX(CASE WHEN user_group = 'Control' THEN avg_flgp_per_user ELSE NULL END), 0),
#             4) AS did_interaction_effect_flgp,

#             COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN total_users ELSE NULL END), 0) AS total_users_A_B
#         FROM order_metrics
#         GROUP BY entity_id
#     )

#     SELECT 
#         om.entity_id,
#         om.user_group,
#         om.avg_flgp_per_user,
#         om.avg_orders_per_user,
#         om.total_orders,
#         om.total_flgp,
#         om.total_users,
#         dc.did_interaction_effect_orders,

#         ROUND(dc.did_interaction_effect_orders * dc.total_users_A_B, 0) AS impact_on_total_orders,

#         dc.did_interaction_effect_flgp,

#         ROUND(dc.did_interaction_effect_flgp * dc.total_users_A_B, 2) AS impact_on_total_flgp

#     FROM order_metrics om
#     JOIN did_calculations dc 
#         ON om.entity_id = dc.entity_id
#     ORDER BY om.entity_id;

#     """

#     return query

# def impact_estimation(entity, test_1, test_2):

#     query = f"""

#         WITH test_data AS (
#         SELECT
#             entity_id,
#             (case when entity_id like '%GV%' then customer_id else perseus_client_id end) perseus_client_id,
#             CAST(test_id AS INT64) AS test_id,
#             LOWER(test_variant) AS test_variant
#         FROM `fulfillment-dwh-production.cl.dps_test_users`
#         WHERE entity_id = '{entity}'
#         AND test_id IN ({test_1}, {test_2})  -- Specific test IDs
#     ),

#     pivoted_data AS (
#         SELECT
#             entity_id,
#             perseus_client_id,
#             MAX(CASE WHEN test_id = {test_1} AND lower(test_variant) = "control" THEN 1 ELSE 0 END) AS is_control_A,
#             MAX(CASE WHEN test_id = {test_1} AND lower(test_variant) LIKE '%variation%' THEN 1 ELSE 0 END) AS is_variation_A,
#             MAX(CASE WHEN test_id = {test_2} AND lower(test_variant) = "control" THEN 1 ELSE 0 END) AS is_control_B,
#             MAX(CASE WHEN test_id = {test_2} AND lower(test_variant) LIKE '%variation%' THEN 1 ELSE 0 END) AS is_variation_B
#         FROM test_data
#         GROUP BY entity_id, perseus_client_id
#     ),

#     classified_users AS (
#         SELECT
#             entity_id,
#             perseus_client_id,
#             CASE
#                 WHEN is_control_A = 1 AND is_control_B = 1 THEN 'Control'
#                 WHEN is_variation_A = 1 AND is_control_B = 1 THEN 'A Only'
#                 WHEN is_control_A = 1 AND is_variation_B = 1 THEN 'B Only'
#                 WHEN is_variation_A = 1 AND is_variation_B = 1 THEN 'A+B'
#             END AS user_group
#         FROM pivoted_data
#         WHERE NOT (is_control_A = 1 AND is_variation_A = 1)  
#         AND NOT (is_control_B = 1 AND is_variation_B = 1)
#     ),

#     user_metrics AS (
#         SELECT 
#             (case when m.entity_id like '%GV%' then m.customer_id else m.perseus_client_id end) perseus_client_id,
#             m.entity_id,
#             COUNT(DISTINCT m.order_id) AS total_orders,
#             SUM(m.fully_loaded_gross_profit_eur) AS flgp_eur
#         FROM `fulfillment-dwh-production.cl.dps_test_orders` m
#         WHERE entity_id = '{entity}'
#         AND test_id in ({test_1}, {test_2})  -- Orders only during the test period
#         GROUP BY perseus_client_id, m.entity_id
#     ),

#     order_metrics AS (
#         SELECT
#             cu.entity_id,
#             cu.user_group,
#             AVG(COALESCE(uo.total_orders, 0)) AS avg_orders_per_user,
#             AVG(COALESCE(uo.flgp_eur, 0)) AS avg_flgp_per_user,
#             SUM(COALESCE(uo.total_orders, 0)) AS total_orders,
#             SUM(COALESCE(uo.flgp_eur, 0)) AS total_flgp,
#             COUNT(DISTINCT cu.perseus_client_id) AS total_users
#         FROM classified_users cu
#         LEFT JOIN user_metrics uo  
#             ON cu.perseus_client_id = uo.perseus_client_id 
#             AND cu.entity_id = uo.entity_id
#         WHERE cu.user_group IS NOT NULL
#         GROUP BY cu.entity_id, cu.user_group
#     ),

#     did_calculations AS (
#         SELECT 
#             entity_id,

#             ROUND(
#                 COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN avg_orders_per_user ELSE NULL END), 0) 
#             - COALESCE(MAX(CASE WHEN user_group = 'A Only' THEN avg_orders_per_user ELSE NULL END), 0)
#             - COALESCE(MAX(CASE WHEN user_group = 'B Only' THEN avg_orders_per_user ELSE NULL END), 0)
#             + COALESCE(MAX(CASE WHEN user_group = 'Control' THEN avg_orders_per_user ELSE NULL END), 0),
#             4) AS did_interaction_effect_orders,

#             ROUND(
#                 COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN avg_flgp_per_user ELSE NULL END), 0) 
#             - COALESCE(MAX(CASE WHEN user_group = 'A Only' THEN avg_flgp_per_user ELSE NULL END), 0)
#             - COALESCE(MAX(CASE WHEN user_group = 'B Only' THEN avg_flgp_per_user ELSE NULL END), 0)
#             + COALESCE(MAX(CASE WHEN user_group = 'Control' THEN avg_flgp_per_user ELSE NULL END), 0),
#             4) AS did_interaction_effect_flgp,

#             COALESCE(MAX(CASE WHEN user_group = 'A+B' THEN total_users ELSE NULL END), 0) AS total_users_A_B
#         FROM order_metrics
#         GROUP BY entity_id
#     )

#     SELECT 
#         om.entity_id,
#         om.user_group,
#         om.avg_flgp_per_user,
#         om.avg_orders_per_user,
#         om.total_orders,
#         om.total_flgp,
#         om.total_users,
#         dc.did_interaction_effect_orders,

#         ROUND(dc.did_interaction_effect_orders * dc.total_users_A_B, 0) AS impact_on_total_orders,

#         dc.did_interaction_effect_flgp,

#         ROUND(dc.did_interaction_effect_flgp * dc.total_users_A_B, 2) AS impact_on_total_flgp

#     FROM order_metrics om
#     JOIN did_calculations dc 
#         ON om.entity_id = dc.entity_id
#     ORDER BY om.entity_id;

#     """

#     return query


# def impact_estimation(entity, test_1, test_2):

#     query = f"""

#             --------------------------------------------------------------------------------
#         -- Step 1: Build up the data with subgroups, pivot them, compute naive/corrected
#         --         differences for both A and B (Orders and FLGP), and store in 'final'
#         --------------------------------------------------------------------------------
#         WITH

#         -- 1) Identify user assignments for test A (ID=68) and test B (ID=69)
#         test_data AS (
#             SELECT
#                 entity_id,
#                 perseus_client_id,
#                 CAST(test_id AS INT64) AS test_id,
#                 LOWER(test_variant) AS test_variant
#             FROM `fulfillment-dwh-production.cl.dps_test_users`
#             WHERE entity_id = '{entity}'
#             AND test_id IN ({test_1}, {test_2})
#         ),

#         -- 2) Pivot each user's assignment to A=Control/Treatment, B=Control/Treatment
#         pivoted_data AS (
#             SELECT
#                 entity_id,
#                 perseus_client_id,
#                 MAX(CASE WHEN test_id = {test_1} AND test_variant = 'control' THEN 1 ELSE 0 END) AS is_A_control,
#                 MAX(CASE WHEN test_id = {test_1} AND test_variant LIKE '%variation%' THEN 1 ELSE 0 END) AS is_A_treatment,
#                 MAX(CASE WHEN test_id = {test_2} AND test_variant = 'control' THEN 1 ELSE 0 END) AS is_B_control,
#                 MAX(CASE WHEN test_id = {test_2} AND test_variant LIKE '%variation%' THEN 1 ELSE 0 END) AS is_B_treatment
#             FROM test_data
#             GROUP BY entity_id, perseus_client_id
#         ),

#         -- 3) Classify each user into exactly one of the 4 subgroups (Control, A Only, B Only, A+B)
#         classified_users AS (
#             SELECT
#                 entity_id,
#                 perseus_client_id,
#                 CASE
#                     WHEN is_A_control = 1    AND is_B_control = 1    THEN 'Control'
#                     WHEN is_A_treatment = 1  AND is_B_control = 1    THEN 'A Only'
#                     WHEN is_A_control = 1    AND is_B_treatment = 1  THEN 'B Only'
#                     WHEN is_A_treatment = 1  AND is_B_treatment = 1  THEN 'A+B'
#                     ELSE NULL
#                 END AS subgroup
#             FROM pivoted_data
#             WHERE NOT (is_A_control = 1 AND is_A_treatment = 1)  
#             AND NOT (is_B_control = 1 AND is_B_treatment = 1)
#         ),

#         -- 4) Get user-level Orders & FLGP
#         user_metrics AS (
#             SELECT
#                 perseus_client_id,
#                 entity_id,
#                 COUNT(DISTINCT order_id) AS total_orders,
#                 SUM(fully_loaded_gross_profit_eur) AS total_flgp
#             FROM `fulfillment-dwh-production.cl.dps_test_orders`
#             WHERE entity_id = '{entity}'
#             AND test_id IN ({test_1}, {test_2})
#             GROUP BY perseus_client_id, entity_id
#         ),

#         -- 5) Summaries for each subgroup: total users, total orders, total flgp, average orders/flgp
#         subgroup_metrics AS (
#             SELECT
#                 cu.entity_id,
#                 cu.subgroup,
#                 COUNT(DISTINCT cu.perseus_client_id)             AS subgroup_user_count,
#                 SUM(COALESCE(um.total_orders, 0))                AS subgroup_total_orders,
#                 SUM(COALESCE(um.total_flgp, 0))                  AS subgroup_total_flgp,
#                 AVG(COALESCE(um.total_orders, 0))                AS subgroup_avg_orders_per_user,
#                 AVG(COALESCE(um.total_flgp, 0))                  AS subgroup_avg_flgp_per_user
#             FROM classified_users cu
#             LEFT JOIN user_metrics um
#                 ON cu.perseus_client_id = um.perseus_client_id
#             AND cu.entity_id = um.entity_id
#             WHERE cu.subgroup IS NOT NULL
#             GROUP BY cu.entity_id, cu.subgroup
#         ),

#         -- 6) Pivot subgroups into columns
#         pivoted_subgroups AS (
#             SELECT
#                 entity_id,

#                 COALESCE(MAX(CASE WHEN subgroup = 'Control' THEN subgroup_user_count END),0) AS control_user_count,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A Only'  THEN subgroup_user_count END),0) AS a_only_user_count,
#                 COALESCE(MAX(CASE WHEN subgroup = 'B Only'  THEN subgroup_user_count END),0) AS b_only_user_count,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A+B'     THEN subgroup_user_count END),0) AS a_b_user_count,

#                 COALESCE(MAX(CASE WHEN subgroup = 'Control' THEN subgroup_avg_orders_per_user END),0) AS control_avg_orders,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A Only'  THEN subgroup_avg_orders_per_user END),0) AS a_only_avg_orders,
#                 COALESCE(MAX(CASE WHEN subgroup = 'B Only'  THEN subgroup_avg_orders_per_user END),0) AS b_only_avg_orders,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A+B'     THEN subgroup_avg_orders_per_user END),0) AS a_b_avg_orders,

#                 COALESCE(MAX(CASE WHEN subgroup = 'Control' THEN subgroup_avg_flgp_per_user END),0) AS control_avg_flgp,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A Only'  THEN subgroup_avg_flgp_per_user END),0) AS a_only_avg_flgp,
#                 COALESCE(MAX(CASE WHEN subgroup = 'B Only'  THEN subgroup_avg_flgp_per_user END),0) AS b_only_avg_flgp,
#                 COALESCE(MAX(CASE WHEN subgroup = 'A+B'     THEN subgroup_avg_flgp_per_user END),0) AS a_b_avg_flgp
#             FROM subgroup_metrics
#             GROUP BY entity_id
#         ),

#         -- 7) 'final' CTE: define all naive/corrected/bias columns for A and B, Orders & FLGP
#         final AS (
#             SELECT
#                 ps.entity_id,

#                 -- Bring in the subgroup counts:
#                 ps.control_user_count,
#                 ps.a_only_user_count,
#                 ps.b_only_user_count,
#                 ps.a_b_user_count,

#                 -- The average metrics for each subgroup:
#                 ps.control_avg_orders,
#                 ps.a_only_avg_orders,
#                 ps.b_only_avg_orders,
#                 ps.a_b_avg_orders,

#                 ps.control_avg_flgp,
#                 ps.a_only_avg_flgp,
#                 ps.b_only_avg_flgp,
#                 ps.a_b_avg_flgp,

#                 -- Summaries for "A=Treatment" vs "A=Control"
#                 (ps.a_b_user_count + ps.a_only_user_count) AS a_treatment_user_count,
#                 (ps.b_only_user_count + ps.control_user_count) AS a_control_user_count,

#                 -- Summaries for "B=Treatment" vs "B=Control"
#                 (ps.a_b_user_count + ps.b_only_user_count) AS b_treatment_user_count,
#                 (ps.a_only_user_count + ps.control_user_count) AS b_control_user_count,

#                 -- Total users
#                 (ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count) AS total_users,

#                 -- =====================================================
#                 -- PART A: ORDERS
#                 -- =====================================================

#                 -- ========== naive_A_diff_orders ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count) = 0
#                     OR (ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     (ps.a_b_avg_orders * ps.a_b_user_count
#                     + ps.a_only_avg_orders * ps.a_only_user_count)
#                     / NULLIF((ps.a_b_user_count + ps.a_only_user_count), 0)
#                     )
#                     -
#                     (
#                     (ps.b_only_avg_orders * ps.b_only_user_count
#                     + ps.control_avg_orders * ps.control_user_count)
#                     / NULLIF((ps.b_only_user_count + ps.control_user_count), 0)
#                     )
#                 END AS naive_a_diff_orders,

#                 -- ========== corrected_A_diff_orders ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     ((ps.a_b_user_count + ps.b_only_user_count)*1.0 
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_b_avg_orders - ps.b_only_avg_orders)
#                     )
#                     +
#                     (
#                     ((ps.a_only_user_count + ps.control_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_only_avg_orders - ps.control_avg_orders)
#                     )
#                 END AS corrected_a_diff_orders,

#                 -- ========== naive_B_diff_orders ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.b_only_user_count) = 0
#                     OR (ps.a_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     (ps.a_b_avg_orders * ps.a_b_user_count
#                     + ps.b_only_avg_orders * ps.b_only_user_count)
#                     / NULLIF((ps.a_b_user_count + ps.b_only_user_count), 0)
#                     )
#                     -
#                     (
#                     (ps.a_only_avg_orders * ps.a_only_user_count
#                     + ps.control_avg_orders * ps.control_user_count)
#                     / NULLIF((ps.a_only_user_count + ps.control_user_count), 0)
#                     )
#                 END AS naive_b_diff_orders,

#                 -- ========== corrected_B_diff_orders ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     ((ps.a_b_user_count + ps.a_only_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_b_avg_orders - ps.a_only_avg_orders)
#                     )
#                     +
#                     (
#                     ((ps.b_only_user_count + ps.control_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.b_only_avg_orders - ps.control_avg_orders)
#                     )
#                 END AS corrected_b_diff_orders,

#                 -- =====================================================
#                 -- PART A: FLGP
#                 -- =====================================================

#                 -- ========== naive_A_diff_flgp ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count) = 0
#                     OR (ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     (ps.a_b_avg_flgp * ps.a_b_user_count
#                     + ps.a_only_avg_flgp * ps.a_only_user_count)
#                     / NULLIF((ps.a_b_user_count + ps.a_only_user_count), 0)
#                     )
#                     -
#                     (
#                     (ps.b_only_avg_flgp * ps.b_only_user_count
#                     + ps.control_avg_flgp * ps.control_user_count)
#                     / NULLIF((ps.b_only_user_count + ps.control_user_count), 0)
#                     )
#                 END AS naive_a_diff_flgp,

#                 -- ========== corrected_A_diff_flgp ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     ((ps.a_b_user_count + ps.b_only_user_count)*1.0 
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_b_avg_flgp - ps.b_only_avg_flgp)
#                     )
#                     +
#                     (
#                     ((ps.a_only_user_count + ps.control_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_only_avg_flgp - ps.control_avg_flgp)
#                     )
#                 END AS corrected_a_diff_flgp,

#                 -- ========== naive_B_diff_flgp ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.b_only_user_count) = 0
#                     OR (ps.a_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     (ps.a_b_avg_flgp * ps.a_b_user_count
#                     + ps.b_only_avg_flgp * ps.b_only_user_count)
#                     / NULLIF((ps.a_b_user_count + ps.b_only_user_count), 0)
#                     )
#                     -
#                     (
#                     (ps.a_only_avg_flgp * ps.a_only_user_count
#                     + ps.control_avg_flgp * ps.control_user_count)
#                     / NULLIF((ps.a_only_user_count + ps.control_user_count), 0)
#                     )
#                 END AS naive_b_diff_flgp,

#                 -- ========== corrected_B_diff_flgp ==========
#                 CASE
#                 WHEN (ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count) = 0
#                 THEN 0
#                 ELSE
#                     (
#                     ((ps.a_b_user_count + ps.a_only_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.a_b_avg_flgp - ps.a_only_avg_flgp)
#                     )
#                     +
#                     (
#                     ((ps.b_only_user_count + ps.control_user_count)*1.0
#                         / NULLIF((ps.a_b_user_count + ps.a_only_user_count + ps.b_only_user_count + ps.control_user_count),0))
#                     * (ps.b_only_avg_flgp - ps.control_avg_flgp)
#                     )
#                 END AS corrected_b_diff_flgp

#                 -- If you also want the "bias_for_a_orders" or "bias_for_b_orders" directly in SQL,
#                 -- you can do "naive - corrected" here. Or you can do that in Python afterward. 
#                 -- Similarly for the "impact_on_total_*" calculations.

#             FROM pivoted_subgroups ps
#         )

#         --------------------------------------------------------------------------------
#         -- Step 2: Filter out any entity_id that doesn't have all 4 subgroups
#         --------------------------------------------------------------------------------
#         SELECT
#         *
#         FROM final
#         WHERE a_only_user_count    > 0
#         AND b_only_user_count    > 0
#         AND control_user_count   > 0
#         AND a_b_user_count       > 0
#         ORDER BY entity_id;



#     """

#     return query




In [ ]:
# import os
# import pandas as pd

# # Define file names for checkpoints
# checkpoint_csv = 'results_checkpoint.csv'
# checkpoint_index_file = 'checkpoint_index.txt'

# # Prepare test_tuples as before
# if {'entity_id', 'test_id_1', 'test_id_2'}.issubset(impact_bias.columns):
#     combined = impact_bias[['entity_id', 'test_id_1', 'test_id_2']].drop_duplicates()
#     entity_list = combined['entity_id'].unique().tolist()
#     test_tuples = combined.apply(lambda row: (row['entity_id'], row['test_id_1'], row['test_id_2']), axis=1).tolist()
# else:
#     entity_list = []
#     test_tuples = []

# # Load checkpoint index if exists, otherwise start at 0
# start_index = 0
# if os.path.exists(checkpoint_index_file):
#     with open(checkpoint_index_file, 'r') as f:
#         try:
#             start_index = int(f.read().strip())
#         except ValueError:
#             start_index = 0

# # Load previous results if available, else create empty DataFrame
# if os.path.exists(checkpoint_csv):
#     results_df = pd.read_csv(checkpoint_csv)
# else:
#     results_df = pd.DataFrame()

# # Process each test tuple starting from the saved index
# for i, (entity, test_1, test_2) in enumerate(test_tuples):
#     # Skip tuples already processed
#     if i < start_index:
#         continue
#     try:
#         # Run the query and convert to DataFrame
#         query_result = client.query(impact_estimation(entity, test_1, test_2)).to_dataframe()

#         # Add test columns for tracking
#         query_result["test_id_1"] = test_1
#         query_result["test_id_2"] = test_2

#         # Append new results to the existing DataFrame
#         results_df = pd.concat([results_df, query_result], ignore_index=True)

#         # Optionally, print progress
#         print(f"Processed tuple {i+1} of {len(test_tuples)}")

#         # Save a checkpoint every 10 iterations (adjust as needed)
#         if (i + 1) % 10 == 0:
#             results_df.to_csv(checkpoint_csv, index=False)
#             with open(checkpoint_index_file, 'w') as f:
#                 f.write(str(i + 1))
#             print(f"Checkpoint saved at iteration {i+1}")

#     except Exception as e:
#         print(f"Error at iteration {i}: {e}")
#         # Save the current progress before breaking or decide to continue.
#         results_df.to_csv(checkpoint_csv, index=False)
#         with open(checkpoint_index_file, 'w') as f:
#             f.write(str(i))
#         break

# # After processing, you may want to remove the checkpoint file
# # os.remove(checkpoint_index_file)

# # Final results are in results_df
